In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

-----------------------------

In [27]:
tail_numbers = test['Tail_Number']
tail_alpha = []

for tail_number in tail_numbers:
    match = re.search(r'\d([A-Za-z]+)$', tail_number)
    if match:
        tail_alpha.append(match.group(1))
    else:
        tail_alpha.append(None)
test['tail_alpha'] =  tail_alpha


In [28]:
IATA_tail = {}
for i in test['Carrier_Code(IATA)'].unique():
    for j in test['tail_alpha'].unique():
        df = test[(test['tail_alpha'] == j) & (test['Carrier_Code(IATA)'] == i)]
        if df.shape[0] > 0:
            if len(df['Carrier_ID(DOT)'].unique()) >= 3:
                IATA_tail[i] = j
            else:
                fill = df['Carrier_ID(DOT)'].max()
                test.loc[df.index, 'Carrier_ID(DOT)'] = fill



------------------------------------------

In [29]:
pairs = list(zip(test['Airline'], test['Carrier_ID(DOT)']))

unique_pairs = {}
for pair in pairs:
    if not any(pd.isna(p) for p in pair):
        unique_pairs[pair] = None

unique_pairs_list = list(unique_pairs.keys())


air_dict = {}
for i in unique_pairs_list:
    air_dict[i[0]] = i[1]

In [30]:
for index, row in test.iterrows():
    if pd.isna(row['Carrier_ID(DOT)']):
        airline = row['Airline']
        if airline in air_dict:
            test.loc[index, 'Carrier_ID(DOT)'] = air_dict[airline]


In [31]:
test['Carrier_ID(DOT)'].isnull().sum()
# 11883

3698

------------------------------------------

In [32]:
iata_alph = {}
for index, row in test.iterrows():
    if pd.isna(row['Carrier_ID(DOT)']):
        pass
    elif pd.isna(row["Carrier_Code(IATA)"]):
        pass
    elif pd.isna(row['tail_alpha']):
        pass
    else:
        if row["Carrier_Code(IATA)"] not in iata_alph:
            iata_alph[(row["Carrier_Code(IATA)"], row['tail_alpha'])] = row['Carrier_ID(DOT)']
        else:
            pass

In [33]:
for index, row in test.iterrows():
    if pd.isna(row['Carrier_ID(DOT)']):
        iata = row['Carrier_Code(IATA)']
        alpha = row['tail_alpha']
        if (iata, alpha) in iata_alph:
            dot = iata_alph[(iata, alpha)]
            test.at[index, 'Carrier_ID(DOT)'] = dot
    else:
        pass


In [34]:
test['Carrier_ID(DOT)'].mode().iloc[0]

19393.0

In [35]:
for index, row in test.iterrows():
    if pd.isnull(row['Carrier_ID(DOT)']):
        test.at[index, 'Carrier_ID(DOT)'] = test['Carrier_ID(DOT)'].mode().iloc[0]

In [36]:
test.to_csv('test.csv', index=False)

In [37]:
test

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,tail_alpha
0,TEST_000000,12,16,1156.0,NaN,0,0,IAH,12266,Texas,SAT,14683,Texas,191.0,United Air Lines Inc.,UA,19977.0,N79402,None
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,ATL,10397,NaN,746.0,Delta Air Lines Inc.,DL,19790.0,N3765,None
2,TEST_000002,3,6,1600.0,1915.0,0,0,ORD,13930,Illinois,LGA,12953,New York,733.0,United Air Lines Inc.,UA,19977.0,N413UA,UA
3,TEST_000003,5,18,1920.0,2045.0,0,0,OAK,13796,California,LAX,12892,California,337.0,Southwest Airlines Co.,WN,19393.0,N905WN,WN
4,TEST_000004,7,7,1915.0,2152.0,0,0,FLL,11697,Florida,LAX,12892,California,2343.0,JetBlue Airways,B6,20409.0,N945JT,JT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TEST_999995,6,2,NaN,2240.0,0,0,LAX,12892,California,DEN,11292,Colorado,862.0,Southwest Airlines Co.,WN,19393.0,N720WN,WN
999996,TEST_999996,6,8,600.0,648.0,0,0,BUF,10792,New York,ORD,13930,Illinois,473.0,United Air Lines Inc.,UA,19977.0,N401UA,UA
999997,TEST_999997,6,11,1710.0,1907.0,0,0,IAD,12264,NaN,DTW,11433,Michigan,383.0,Mesa Airlines Inc.,UA,20378.0,N510MJ,MJ
999998,TEST_999998,11,17,NaN,500.0,0,0,SAN,14679,California,BOS,10721,Massachusetts,2588.0,JetBlue Airways,B6,20409.0,N986JB,JB
